In [1]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image

import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import os
import NN
import deep_networks


In [3]:
# -*- coding: utf-8 -*-
"""
Created on Tue Oct  1 09:10:31 2019

@author: samy
"""

from datamaestro import prepare_dataset
from torch.utils.data import Dataset, DataLoader



class DatasetMnist(Dataset):
    def __init__(self,step, transform=None):
        ds = prepare_dataset("com.lecun.mnist")
        self.data = ds.files[step+"/images"].data()
        self.labels = ds.files[step+"/labels"].data()
        self.transform = transform
    def __getitem__(self,index):
        if self.transform:
            x = self.transform(self.data[index])
        else:
            x=self.data[index]
        return x,self.labels[index]
        
    def __len__(self):
        return len(self.data)
        



In [4]:
transform = transforms.Compose([transforms.ToTensor(),  transforms.Normalize((0,  ), (            1,))])
data_train=DataLoader(DatasetMnist("train",transform),shuffle=True,batch_size=20)
data_test=DataLoader(DatasetMnist("test",transform),shuffle=True,batch_size=20)

In [5]:

for features,label in data_train:
    print(features[0].shape)
    break

torch.Size([1, 28, 28])


#### nous effectuons une classification donc nous aurons les 10 labels de different chiffre

In [23]:
image_size=features.shape[2]*features.shape[3]
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

for nb_layers in range(1,101,10):
    #print(nb_layers)
    plain_Net=deep_networks.Plain_deep(image_size,10,nb_layers,hidden_shape=71)
    highway_Net=deep_networks.Highway_deep(image_size,10,nb_layers,hidden_shape=50)
    optimiser_Plain=torch.optim.SGD(plain_Net.parameters(),lr=1e-3)
    optimiser_Highway=torch.optim.SGD(highway_Net.parameters(),lr=1e-3)
    distance_Plain=torch.nn.CrossEntropyLoss()
    distance_Highway=torch.nn.CrossEntropyLoss()
    progress_Plain=[]
    progress_Highway=[]
    for epoch in range(50):
        #print(epoch)
        for features,label in data_train:
            with torch.no_grad():
                label=label.type(torch.LongTensor)
            images_flatten=features.reshape((features.shape[0],-1))
            scores_Plain=plain_Net(images_flatten)
            scores_Highway=highway_Net(images_flatten)
            loss_Plain=distance_Plain(scores_Plain,label)
            loss_Highway=distance_Highway(scores_Highway,label)
            
            ########Nous allons effectuer le backward###########
            optimiser_Highway.zero_grad()
            loss_Highway.backward()
            optimiser_Highway.step()
            #progress_Plain.append(loss_Plain)
            #progress_Highway.append(loss_Highway)
            optimiser_Plain.zero_grad()
            loss_Plain.backward()
            optimiser_Plain.step()
        
        writer.add_scalar('Plain'+str(nb_layers), loss_Plain, epoch)
        writer.add_scalar('Highway'+str(nb_layers), loss_Highway, epoch)
        #print("plain",loss_Plain)
        #print("highway",loss_Highway)
            
writer.close()
            
            
            

            

KeyboardInterrupt: 

With GPU

In [6]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
savepath= os.path.join('model', 'progression')
class State:
    def __init__(self,model,optim):
        self.model=model
        self.optim=optim
        self.epoch,self.iteration=0,0

In [ ]:
ITERATIONS=100
image_size=features.shape[2]*features.shape[3]
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

for nb_layers in range(1,101,10):
    plain_Net=deep_networks.Plain_deep(image_size,10,nb_layers,hidden_shape=71)
    highway_Net=deep_networks.Highway_deep(image_size,10,nb_layers,hidden_shape=50)
    optimiser_Plain=torch.optim.SGD(plain_Net.parameters(),lr=1e-3)
    optimiser_Highway=torch.optim.SGD(highway_Net.parameters(),lr=1e-3)
    distance_Plain=torch.nn.CrossEntropyLoss()
    distance_Highway=torch.nn.CrossEntropyLoss()
    
    
    state_plain=State(plain_Net.to(device),optimiser_Plain)
    state_highway=State(highway_Net.to(device),optimiser_Highway)
    
    for epoch in range(0,100):
        affiche=True
        for features,label in data_train:
            with torch.no_grad():
                label=label.type(torch.LongTensor)
                label=label.to(device)
            images_flatten=features.reshape((features.shape[0],-1))
            images_flatten=images_flatten.to(device)
            scores_Plain=state_plain.model.forward(images_flatten)
            scores_Highway=state_highway.model.forward(images_flatten)
            
            loss_Plain=distance_Plain(scores_Plain,label)
            loss_Highway=distance_Highway(scores_Highway,label)
            
            ########Nous allons effectuer le backward###########
            
            state_plain.optim.zero_grad()
            loss_Plain.backward()
            state_plain.optim.step()
            #progress_Plain.append(loss_Plain)
            #progress_Highway.append(loss_Highway)
            
            state_highway.optim.zero_grad()
            loss_Highway.backward()
            state_highway.optim.step()
        
        writer.add_scalar('Plain'+str(nb_layers), loss_Plain, epoch)
        writer.add_scalar('Highway'+str(nb_layers), loss_Highway, epoch)
            
            
            
    print("plain",loss_Plain)
    print("highway",loss_Highway)

plain tensor(1.6257, device='cuda:0', grad_fn=<NllLossBackward>)
highway tensor(1.6343, device='cuda:0', grad_fn=<NllLossBackward>)
plain tensor(1.8584, device='cuda:0', grad_fn=<NllLossBackward>)
highway tensor(1.7153, device='cuda:0', grad_fn=<NllLossBackward>)
plain tensor(2.3090, device='cuda:0', grad_fn=<NllLossBackward>)
highway tensor(1.7131, device='cuda:0', grad_fn=<NllLossBackward>)
plain tensor(2.2996, device='cuda:0', grad_fn=<NllLossBackward>)
highway tensor(1.8255, device='cuda:0', grad_fn=<NllLossBackward>)


In [19]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

#### On peut voir apres cette execution que le Highway Network arrive a faire correctement la descente de gradient meme avec plusieur couches ce qui n'est pas le cas de l'algo plain